# To test the graph search with a different approach
## Pick a single node which the cluster will refer to using a refers to relation

In [1]:
# This code will load the clusters from a text file, and merge the nodes in the cluster together
import ast
from resources import node_creation
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace
import os
from tqdm import tqdm

base_path = 'Data/cluster_data/output_all/'
cluster_dirs = os.listdir(base_path)
cluster_docs = [base_path + dir +'/event_clusters.txt' for dir in cluster_dirs if os.path.isdir(base_path + dir)]

In [4]:
#Construct a dictionary that merges every cluster, since a mention can belong to multiple clusters
graph = Graph()
graph.parse('Data/graphs/eag_complete_new.ttl')
data = pd.read_csv('Data/final_data_with_predictions.csv') #Load the original dataset
owl = Namespace("http://www.w3.org/2002/07/owl#")
ex = Namespace("http://www.example.org/")

all_clusters = {} #All the mentions

for doc in tqdm(cluster_docs):

    with open(doc) as f: #Open the generated cluster file
        cluster_doc = f.readlines()

    for cluster in cluster_doc:
        cluster_list = [] #the values of the key

        if cluster.startswith('['):
            cluster = ast.literal_eval(cluster)

            if len(cluster) != 1:
                key = None


                for mention in cluster:

                    mention = mention.split('_')

                    sentence = data[(data['URI'] == mention[1]) & (data['Sentence_num'] == int(mention[2]))]['Sentence'].values[0]
                    sentence_uri = node_creation('', sentence, base_add='/sentence') #Generate the URI for the sentence


                    #determine subject or object
                    if data[(data['URI'] == mention[1]) & (data['Sentence_num'] == int(mention[2]))]['Subject'].values[0] == mention[0]:
                        mention_uri = node_creation('', mention[0] + str(sentence_uri), base_add='/subject')

                    else:
                        mention_uri = node_creation('', mention[0] + str(sentence_uri), base_add='/object')

                    if key == None:
                        key = mention_uri
                    else:
                        cluster_list.append(mention_uri)

                all_clusters[key] = cluster_list

100%|██████████| 86/86 [01:43<00:00,  1.21s/it]


In [6]:
for key in all_clusters:
    for value in all_clusters[key]:
        graph.add((value, ex.refers_to, key))

http://kflow.eurecom.fr/subject/dee83279-b063-57e4-a80b-3c337a1daeac
http://kflow.eurecom.fr/object/8735a2cc-1c2c-5dd0-91f2-0ae2f028d1a6
http://kflow.eurecom.fr/object/7630174c-c6b0-5d56-852e-2cf1e6cd6c0b
http://kflow.eurecom.fr/subject/50e5d007-c42f-5331-a944-682b200fd544
http://kflow.eurecom.fr/object/1f581a99-982f-5d46-a5fd-8d2103c45844
http://kflow.eurecom.fr/subject/c8de070f-c147-51bd-a0ea-5f38150db9b6
http://kflow.eurecom.fr/subject/8f00f4a8-e763-53fc-a73d-19ef63d7b042
http://kflow.eurecom.fr/subject/93dce23a-5762-585f-a5be-7de2c6c0e2c5
http://kflow.eurecom.fr/subject/fc2d591a-83e5-5fcf-a6fe-db43064c3db7
http://kflow.eurecom.fr/subject/256eed2b-7384-5ec9-aa28-d22b391250f3
http://kflow.eurecom.fr/object/0b1dee77-79d4-53e2-9968-f83d7792be06
http://kflow.eurecom.fr/object/4cd3cd7e-e5cc-58a7-bf79-d238c903f735
http://kflow.eurecom.fr/object/bca33cbc-5e15-5041-8cdb-91b5c7d6ff93
http://kflow.eurecom.fr/object/17a83566-0ca2-5d03-b1df-92e3c348a4dc
http://kflow.eurecom.fr/subject/736c3fef-